# Projeto 1 - Ciência dos Dados

Nome: Filippo Ferraro

Nome: Nicolas Byung Kwan Cho

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [2]:
import os

filename = 'iPad Pro.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo iPad Pro.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [3]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevância
0,novo ipad air destruidor demais. não precisa g...,1
1,"@beniiciojr @elizabethrsic ih, facil é só eu p...",0
2,o negócio é comprar o ipad air preto e dizer q...,1
3,"vídeo feito com realidade aumentada, alguma su...",1
4,"@drudu mas migo, n é meio parecido com o últim...",1


In [4]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevância
0,rt @guiqis: novo ipad air: layout do ipad pro ...,1
1,novas cores do novo airpad air.\n\n- novo proc...,1
2,eu só quero um ipad pro,0
3,usb-c estará presente no novo ipad air.\n\num ...,1
4,"o design é baseado no ipad pro de 2018, soment...",1


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

O iPad Pro é um dispositivo do tipo tablet desenvolvido pela Apple. Por ser um produto de uma marca renomada e apresentar alto desempeno, o iPad Pro não é um dispositivo acessível a todos.
Portanto, tweets que mencionavam o preço do produto, design, utilidade e especificações técnicas, foram considerados relevantes.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;@\n@]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed
    

In [6]:
#limpando os tweets
train.Treinamento.astype(str)
train.Treinamento = train.Treinamento.apply(cleanup)
train

,Treinamento,Relevância
0,novo ipad air destruidor demais não precisa ga...,1
1,beniiciojr elizabethrsic ih facil é só eu parc...,0
2,o negócio é comprar o ipad air preto e dizer q...,1
3,vídeo feito com realidade aumentada alguma sug...,1
4,drudu mas migo n é meio parecido com o último ...,1
5,meu sonho um ipad pro alguem pfv me da,1
6,tivemos apple watch ipad e processador mas sem...,0
7,danielsatann espera para a semana o novo ipad ...,1
8,procreate é um programa de desenhar no ipad pr...,1
9,rt jukirigoyen oi gente de blu e região estou ...,0


In [7]:
#colocando as palavras dos tweets relevantes em uma serie pd
tweets_relevantes = train.loc[train.Relevância == 1, : ]
todos_tweets_relevantes = ''
for e in tweets_relevantes.Treinamento:
    todos_tweets_relevantes += e

lista_tweets_relevantes = todos_tweets_relevantes.split()
series_tweets_relevantes = pd.Series(lista_tweets_relevantes)
series_tweets_relevantes

0             novo
1             ipad
2              air
3       destruidor
4           demais
5              não
6          precisa
7           gastar
8              com
9                o
10             pro
11             vai
12             ser
13             meu
14        proximoo
15         negócio
16               é
17         comprar
18               o
19            ipad
20             air
21           preto
22               e
23           dizer
24               q
25               é
26              um
27            ipad
28             pro
29              de
           ...    
5962        preços
5963            na
5964         apple
5965        brasil
5966           não
5967           foi
5968          nada
5969        bonito
5970             o
5971          ipad
5972           pro
5973         agora
5974         custa
5975            10
5976           mil
5977            na
5978        versão
5979          mais
5980        barata
5981          aqui
5982      😂🇧🇷…ipad
5983        

In [19]:
#suavizacao de laplace
palavras_desconhecidas = 1000000
bonus = 2

In [33]:
#frequencias relativas das palavras em tweets relevantes (com suavizacao de laplace)
freq_tweets_relevantes = series_tweets_relevantes.value_counts() 
total_relevantes = sum(freq_tweets_relevantes) + palavras_desconhecidas*bonus

freq_rel_tweets_relevantes = (freq_tweets_relevantes+bonus) / total_relevantes
freq_rel_tweets_relevantes

ipad                         0.000185
pro                          0.000134
o                            0.000132
e                            0.000079
que                          0.000069
de                           0.000067
um                           0.000065
air                          0.000050
do                           0.000048
é                            0.000048
com                          0.000045
eu                           0.000040
mais                         0.000038
a                            0.000035
não                          0.000035
apple                        0.000034
pra                          0.000030
novo                         0.000026
no                           0.000024
tem                          0.000022
mas                          0.000019
em                           0.000018
se                           0.000017
comprar                      0.000017
q                            0.000016
design                       0.000016
me          

In [34]:
#colocando as palavras dos tweets irrelevantes em uma serie pd
tweets_irrelevantes = train.loc[train.Relevância == 0, : ]
todos_tweets_irrelevantes = ''
for e in tweets_irrelevantes.Treinamento:
    todos_tweets_irrelevantes += e

lista_tweets_irrelevantes = todos_tweets_irrelevantes.split()
series_tweets_irrelevantes = pd.Series(lista_tweets_irrelevantes)
series_tweets_irrelevantes

0                         beniiciojr
1                      elizabethrsic
2                                 ih
3                              facil
4                                  é
5                                 só
6                                 eu
7                           parcelar
8                                 um
9                              carro
10                                ou
11                            querer
12                           comprar
13                                um
14                              ipad
15                               pro
16      d0sa9d8sa76drafgdhajstivemos
17                             apple
18                             watch
19                              ipad
20                                 e
21                       processador
22                               mas
23                               sem
24                            iphone
25                            efeito
26                                da
2

In [35]:
#frequencias relativas das palavras em tweets irrelevantes (com suavizacao de laplace)
freq_tweets_irrelevantes = series_tweets_irrelevantes.value_counts() 
total_irrelevantes = sum(freq_tweets_irrelevantes) + palavras_desconhecidas*bonus

freq_rel_tweets_irrelevantes = (freq_tweets_irrelevantes+bonus) / total_irrelevantes
freq_rel_tweets_irrelevantes

ipad                             0.000112
pro                              0.000096
e                                0.000076
o                                0.000063
de                               0.000059
do                               0.000043
eu                               0.000042
que                              0.000042
a                                0.000041
um                               0.000041
apple                            0.000039
com                              0.000033
air                              0.000030
no                               0.000022
é                                0.000022
da                               0.000021
iphone                           0.000019
mais                             0.000018
mas                              0.000018
não                              0.000018
meu                              0.000017
pra                              0.000017
watch                            0.000016
uma                              0

In [12]:
################################## célula desnecessária
#definindo um conjunto universo
universo = todos_tweets_relevantes + todos_tweets_irrelevantes
serie_universo = pd.Series(universo.split())
freq_rel_universo = serie_universo.value_counts(True)
freq_rel_universo
##################################

ipad                               0.055183
pro                                0.042527
o                                  0.036013
e                                  0.028569
de                                 0.023078
que                                0.020473
um                                 0.019542
do                                 0.016750
eu                                 0.014982
air                                0.014610
com                                0.014238
a                                  0.013866
apple                              0.013307
é                                  0.012749
mais                               0.010236
não                                0.009492
pra                                0.008375
no                                 0.008282
tem                                0.006514
mas                                0.006514
novo                               0.006235
da                                 0.006049
meu                             

In [39]:
def classificador(tweet):
    lista_palavras = tweet.split()
    
    P_tweet_dado_relevante = 1
    P_tweet_dado_irrelevante = 1
    
    for palavra in lista_palavras:
        if palavra in freq_rel_tweets_relevantes:
            P_tweet_dado_relevante *= freq_rel_tweets_relevantes[palavra]
        else:
            P_tweet_dado_relevante *= bonus/total_relevantes
            
        if palavra in freq_rel_tweets_irrelevantes:
            P_tweet_dado_irrrelevante *= freq_rel_tweets_irrelevantes[palavra]
        else:
            P_tweet_dado_irrelevante *= bonus/total_irrelevantes
            
    P_relevante = len(tweets_relevantes)/len(train)
    P_irrelevante = len(tweets_irrelevantes)/len(train)
    
    P_relevante_dado_tweet = P_tweet_dado_relevante * P_relevante
    P_irrelevante_dado_tweet = P_tweet_dado_irrelevante * P_irrelevante
    
    if P_relevante_dado_tweet > P_irrelevante_dado_tweet:
        return 1
    else:
        return 2

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**